# 数据变换 Transforms

通常情况下，直接加载的原始数据并不能直接送入神经网络进行训练，此时我们需要对其进行数据预处理。

`mindspore.dataset`提供了面向图像、文本等不同数据类型的Transforms，同时也支持使用Lambda函数。

所有的Transforms均可通过`map`方法传入，实现对指定数据列的处理,下面分别对其进行介绍。

In [1]:
import numpy as np
from PIL import Image
from download import download
from mindspore.dataset import transforms, vision, text
from mindspore.dataset import GeneratorDataset, MnistDataset

## Transforms 模块

`mindspore.dataset.transforms`模块支持一系列通用Transforms。这里我们以`Compose`为例，介绍其使用方式。

其他Transforms方法：
- OneHot：将标签转换为one-hot编码；
- TypeCast：转换数据类型
- ...

### Compose

`Compose`接收一个数据增强操作序列，然后将其组合成单个数据增强操作。我们仍基于Mnist数据集呈现Transforms的应用效果。

In [2]:
# Download data from open datasets

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/" \
      "notebook/datasets/MNIST_Data.zip"
path = download(url, "./", kind="zip", replace=True)

train_dataset = MnistDataset('MNIST_Data/train')


file_sizes: 100%|██████████████████████████| 10.8M/10.8M [00:00<00:00, 12.1MB/s]
Extracting zip file...
Successfully downloaded / unzipped to ./


In [3]:
image, label = next(train_dataset.create_tuple_iterator())
print(image.shape)

(28, 28, 1)


In [18]:
#创建组合变换
composed = transforms.Compose(
    [
        vision.Rescale(1.0 / 255.0, 0), # 将像素值缩放为0~1
        vision.Normalize(mean=(0.1307,), std=(0.3081,)),
        vision.HWC2CHW() #从高度×宽度×通道-->通道×高度×宽度
    ]
)

In [5]:
train_dataset = train_dataset.map(composed, 'image')
image, label = next(train_dataset.create_tuple_iterator())
print(image.shape)

(1, 28, 28)


## Vision 模块

`mindspore.dataset.vision`模块提供一系列针对图像数据的Transforms。在Mnist数据处理过程中，使用了`Rescale`、`Normalize`和`HWC2CHW`变换。下面对其进行详述。

### Rescale

`Rescale`变换用于调整图像像素值的大小，包括两个参数：

- rescale：缩放因子。
- shift：平移因子。

图像的每个像素将根据这两个参数进行调整，输出的像素值为$output_{i} = input_{i} * rescale + shift$。

这里我们先使用numpy随机生成一个像素值在\[0, 255\]的图像，将其像素值进行缩放。

In [6]:
random_np = np.random.randint(0, 255, (48, 48), np.uint8)
random_image = Image.fromarray(random_np)
print(random_np)

[[105  28 225 ...  42 168  56]
 [201 129 228 ... 201 190 171]
 [142 194   5 ...  94 149  39]
 ...
 [ 56 253 166 ...  23 148 110]
 [225 141  66 ... 218 139  15]
 [203 179  14 ... 125 204 175]]


为了更直观地呈现Transform前后的数据对比，我们使用Transforms的[Eager模式](https://www.mindspore.cn/tutorials/zh-CN/r2.3/advanced/dataset/eager.html)进行演示。首先实例化Transform对象，然后调用对象进行数据处理。

In [7]:
rescale = vision.Rescale(1.0 / 255.0, 0)
rescaled_image = rescale(random_image)
print(rescaled_image)

[[0.41176474 0.10980393 0.882353   ... 0.16470589 0.65882355 0.21960786]
 [0.78823537 0.5058824  0.8941177  ... 0.78823537 0.74509805 0.67058825]
 [0.5568628  0.7607844  0.01960784 ... 0.36862746 0.58431375 0.15294118]
 ...
 [0.21960786 0.9921569  0.6509804  ... 0.09019608 0.5803922  0.43137258]
 [0.882353   0.5529412  0.25882354 ... 0.854902   0.54509807 0.05882353]
 [0.7960785  0.7019608  0.05490196 ... 0.4901961  0.8000001  0.6862745 ]]


可以看到，使用`Rescale`后的每个像素值都进行了缩放。

### Normalize

[Normalize](https://www.mindspore.cn/docs/zh-CN/r2.3/api_python/dataset_vision/mindspore.dataset.vision.Normalize.html)变换用于对输入图像的归一化，包括三个参数：

- mean：图像每个通道的均值。
- std：图像每个通道的标准差。
- is_hwc：bool值，输入图像的格式。
True为(height, width, channel)，False为(channel, height, width)。

图像的每个通道将根据`mean`和`std`进行调整，计算公式为$output_{c} = \frac{input_{c} - mean_{c}}{std_{c}}$，其中 $c$代表通道索引。

In [8]:
normalize = vision.Normalize(mean=(0.1307,), std=(0.3081,))
normalized_image = normalize(rescaled_image)
print(normalized_image)

[[ 0.9122518  -0.06782239  2.4396398  ...  0.11037287  1.7141304
   0.28856817]
 [ 2.1341622   1.2177293   2.4778247  ...  2.1341622   1.9941516
   1.7523152 ]
 [ 1.3831964   2.0450647  -0.3605718  ...  0.77224106  1.472294
   0.07218818]
 ...
 [ 0.28856817  2.7960305   1.688674   ... -0.13146357  1.4595658
   0.9758929 ]
 [ 2.4396398   1.3704681   0.41585052 ...  2.3505423   1.3450117
  -0.23328944]
 [ 2.1596189   1.854141   -0.24601768 ...  1.1668164   2.172347
   1.803228  ]]


### HWC2CHW

`HWC2CHW`变换用于转换图像格式。在不同的硬件设备中可能会对(height, width, channel)或(channel, height, width)两种不同格式有针对性优化。MindSpore设置HWC为默认图像格式，在有CHW格式需求时，可使用该变换进行处理。

这里我们先将前文中`normalized_image`处理为HWC格式，然后进行转换。可以看到转换前后的shape发生了变化。

In [9]:
hwc_image = np.expand_dims(normalized_image, -1)
hwc2chw = vision.HWC2CHW()
chw_image = hwc2chw(hwc_image)
print(hwc_image.shape, chw_image.shape)

(48, 48, 1) (1, 48, 48)


## Text 模块

`mindspore.dataset.text`模块提供一系列针对文本数据的Transforms。与图像数据不同，文本数据需要有分词（Tokenize）、构建词表、Token转Index等操作。这里简单介绍其使用方法。

首先我们定义一段文本，作为待处理的数据，并使用`GeneratorDataset`进行加载。

In [10]:
texts = ['Welcome to Beijing']
test_dataset = GeneratorDataset(texts, 'text')

### PythonTokenizer

分词（Tokenize）操作是文本数据的基础处理方法，MindSpore提供多种不同的Tokenizer。这里我们选择基础的`PythonTokenizer`举例，此Tokenizer允许用户自由实现分词策略。随后我们利用`map`操作将此分词器应用到输入的文本中，对其进行分词。

In [11]:
#先定义分词的规则
def my_tokenizer(content):
    return content.split()

test_dataset = test_dataset.map(text.PythonTokenizer(my_tokenizer))
print(next(test_dataset.create_tuple_iterator()))

[Tensor(shape=[3], dtype=String, value= ['Welcome', 'to', 'Beijing'])]


### Lookup

`Lookup`为词表映射变换，用来将Token转换为Index。

在使用`Lookup`前，需要构造词表，一般可以加载已有的词表，或使用`text.Vocab`生成词表。这里我们选择使用`text.Vocab.from_dataset`方法从数据集中生成词表。

In [12]:
vocab = text.Vocab.from_dataset(test_dataset)

获得词表后我们可以使用`vocab`方法查看词表。

In [13]:
print(vocab.vocab())

{'Beijing': 0, 'Welcome': 1, 'to': 2}


生成词表后，可以使用`lookup` 配合`map`方法进行词表映射变换，将Token转为Index。

In [14]:
test_dataset = test_dataset.map(text.Lookup(vocab))
print(next(test_dataset.create_tuple_iterator()))

[Tensor(shape=[3], dtype=Int32, value= [1, 2, 0])]


## Lambda 自定义数据变换

Lambda函数不需要名字即可定义，表达式会在调用时被求值。

Lambda Transforms可以加载任意定义的Lambda函数，提供足够的灵活度。

在这里，我们首先使用一个简单的Lambda函数，对输入数据乘2

In [15]:
test_dataset = GeneratorDataset([1, 2, 3], 'data', shuffle=False)
test_dataset = test_dataset.map(lambda x: x * 2)
print(list(test_dataset.create_tuple_iterator()))

[[Tensor(shape=[], dtype=Int64, value= 2)], [Tensor(shape=[], dtype=Int64, value= 4)], [Tensor(shape=[], dtype=Int64, value= 6)]]


可以看到`map`传入Lambda函数后，迭代获得数据进行了乘2操作。

也可以定义较复杂的函数，配合Lambda函数实现复杂数据处理：

In [16]:
def func(x):
    return x * x + 2

test_dataset = test_dataset.map(lambda x: func(x))

In [17]:
print(list(test_dataset.create_tuple_iterator()))

[[Tensor(shape=[], dtype=Int64, value= 6)], [Tensor(shape=[], dtype=Int64, value= 18)], [Tensor(shape=[], dtype=Int64, value= 38)]]
